In [1]:
%%capture
!pip install -U "transformers>=4.39.0"
!pip install peft bitsandbytes
!pip install -U "trl>=0.8.3"

In [ ]:
!pip install flash-attn --no-build-isolation

In [2]:
!pip install qwen-vl-utils[decord]==0.0.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 39.0 MB/s eta 0:00:00


In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/content/Normal-10.png",
            },
            {"type": "text", "text": """Given the symptoms: My cough is dry and persistent, and my voice has become slightly raspy. "
    "I feel a bit of discomfort in my chest, but it doesn’t hurt when I take deep breaths. "
    "I get occasional headaches and feel sluggish during the day. I don’t have night sweats, "
    "but my body feels warmer than usual, and based on the X-ray of my chest, detect the chances of "
    "getting tuberculosis and explain why."""},
        ],
    }
]
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

['Based on the symptoms you described, it is likely that you have a condition known as chronic obstructive pulmonary disease (COPD). COPD is a chronic lung disease that causes the airways in the lungs to become inflamed and narrowed, making it difficult for you to breathe. The symptoms you described, such as dry cough, persistent cough, and a feeling of discomfort in the chest, are common in COPD.\nThe X-ray of your chest shows that you have a normal-sized heart and lungs, but there is a small amount of fluid in the pleural space, which is the space between the lungs and the chest wall.']


In [ ]:
"""Based on the symptoms you described, it is likely that you have a c
ondition known as chronic obstructive pulmonary disease (COPD). COPD is a
chronic lung disease that causes the airways in the lungs to become inflamed
and narrowed, making it difficult for you to breathe. The symptoms you described,
such as dry cough, persistent cough, and a feeling of discomfort in the chest,
are common in COPD. The X-ray of your chest shows that you have a normal-sized heart and lungs,
but there is a small
amount of fluid in the pleural space, which is the
space between the lungs and the chest wall."""